In [1]:
!pip install autogluon

In [ ]:
!mkdir -p price_of_books
!wget https://automl-mm-bench.s3.amazonaws.com/machine_hack_competitions/predict_the_price_of_books/Data.zip -O price_of_books/Data.zip
!cd price_of_books && unzip -o Data.zip
!ls price_of_books/Participants_Data

In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import KFold

train = pd.read_excel('./price_of_books/Participants_Data/Data_Train.xlsx')

In [ ]:
kf = KFold(n_splits=3, random_state=1001,shuffle=True)
for i, (train_index, val_index) in enumerate(kf.split(train)):
    trn= train.iloc[train_index].reset_index()
    val= train.iloc[val_index].reset_index()
    
trn = trn.drop(columns=['index'])
val = val.drop(columns=['index'])

val.to_csv('sub_val.csv',index=False)
trn.to_csv('sub_train.csv',index=False)

In [3]:
from autogluon.text import TextPredictor
import pandas as pd

train_data = pd.read_csv('sub_train.csv')
test_data = pd.read_csv('sub_val.csv')
test_data_nolab = test_data.drop(columns=['Price']) 
time_limit = 1 * 60  # set to larger value in your applications
predictor = TextPredictor(label='Price', path='autogluon')
predictor.fit(train_data, time_limit=time_limit)
y_pred = predictor.predict(test_data_nolab)

Global seed set to 123
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type                | Params
----------------------------------------------------------
0 | model             | MultimodalFusionMLP | 109 M 
1 | validation_metric | MeanSquaredError    | 0     
2 | loss_func         | MSELoss             | 0     
----------------------------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
219.565   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 123


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Epoch 0, global step 28: val_rmse reached 0.81566 (best 0.81566), saving model to "/root/autodl-tmp/autogluon/epoch=0-step=28.ckpt" as top 3
Time limit reached. Elapsed time is 0:01:00. Signaling Trainer to stop.


Validating: 0it [00:00, ?it/s]

Epoch 1, global step 30: val_rmse reached 0.83246 (best 0.81566), saving model to "/root/autodl-tmp/autogluon/epoch=1-step=30.ckpt" as top 3


Predicting: 0it [00:00, ?it/s]

In [4]:
from sklearn.metrics import mean_squared_error
import numpy as np
RMSE = np.sqrt(mean_squared_error(test_data['Price'], y_pred))
RMSE

0.6973654451724718